In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download('omw-1.4')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [2]:
from nltk.corpus import stopwords
from nltk.stem.porter import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as matplotlib
%matplotlib inline
pd.options.plotting.backend = "plotly"

In [3]:
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
ps=PorterStemmer()
lem=WordNetLemmatizer()

In [4]:
dataframe=pd.read_excel("tweet_10000_records.xlsx")
dataframe.head()

,Text,Language,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,litigious
1,#BadBunny: Como dos gotas de agua: Joven se di...,es,negative
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,litigious
3,Rwanda is set to host the headquarters of Unit...,en,positive
4,OOPS. I typed her name incorrectly (todayâ€™s ...,en,litigious


In [5]:
Labels=dataframe['Label'].unique().tolist()
sorted_labels=sorted(Labels)
n=list(range(len(sorted_labels)))
mapping=dict(zip(sorted_labels,n))
rev_map=dict(zip(n,sorted_labels))
dataframe['Label']=dataframe['Label'].map(mapping)

In [28]:
dataframe.head(10)

,Text,Language,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,0
1,#BadBunny: Como dos gotas de agua: Joven se di...,es,1
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,0
3,Rwanda is set to host the headquarters of Unit...,en,2
4,OOPS. I typed her name incorrectly (todayâ€™s ...,en,0
5,It sucks for me since I'm focused on the natur...,en,1
6,@en_font Treballar a lâ€™obra a partir dels 19...,ca,1
7,@ShawnTarloff @itsmieu you can also relate thi...,en,3
8,Social Security. Constant political crises dis...,en,1
9,@FilmThePoliceLA A broken rib can puncture a l...,en,1


In [7]:
df=dataframe[['Text','Label']]
df.head()

,Text,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,0
1,#BadBunny: Como dos gotas de agua: Joven se di...,1
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,0
3,Rwanda is set to host the headquarters of Unit...,2
4,OOPS. I typed her name incorrectly (todayâ€™s ...,0


In [8]:
def clean_text(input_tweet):
  text=input_tweet.lower()
  text=re.sub('@[a-zA-Z0-9_]+','',text)
  text=re.sub('#[a-zA-Z0-9_]+','',text)
  text=re.sub(r'http\S+','',text)
  text=re.sub('[^a-zA-Z0-9]',' ',text)
  words=text.split()
  words=[word for word in words if word not in stopwords.words("english")]
  words=[lem.lemmatize(wo) for wo in words]
  #words=[ps.stem(w) for w in words]
  
  words=' '.join(words)
  return words


In [9]:
clean_text("@varma #kondru is another son, 495 is his id number")

'another son 495 id number'

In [79]:
ex=dataframe.head()
ex

,Text,Language,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,en,0
1,#BadBunny: Como dos gotas de agua: Joven se di...,es,1
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,en,0
3,Rwanda is set to host the headquarters of Unit...,en,2
4,OOPS. I typed her name incorrectly (todayâ€™s ...,en,0


In [10]:
df['Text']

0       @Charlie_Corley @Kristine1G @amyklobuchar @Sty...
1       #BadBunny: Como dos gotas de agua: Joven se di...
2       https://t.co/YJNiO0p1JV Flagstar Bank disclose...
3       Rwanda is set to host the headquarters of Unit...
4       OOPS. I typed her name incorrectly (todayâ€™s ...
                              ...                        
9995    @nathanias meanwhile somewhere in america some...
9996    Who are these anti-vaxx â€œwellness warriorsâ€...
9997    @jukegraphics @albertz__ @Luminosity Really wa...
9998    @romeoauerbach Diputado por favor presente una...
9999    Iâ€™m probably going to cut my hair again who ...
Name: Text, Length: 10000, dtype: object

In [11]:
df['cleaned']=df['Text'].apply(lambda row: clean_text(row))


In [13]:
df=df[['Text','cleaned','Label']]
df

,Text,cleaned,Label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,testimony evidence court law state federal mus...,0
1,#BadBunny: Como dos gotas de agua: Joven se di...,como do gotas de agua joven se disfraza de bad...,1
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,flagstar bank discloses data breach impacted 1...,0
3,Rwanda is set to host the headquarters of Unit...,rwanda set host headquarters united nation dev...,2
4,OOPS. I typed her name incorrectly (todayâ€™s ...,oops typed name incorrectly today brave witnes...,0
...,...,...,...
9995,@nathanias meanwhile somewhere in america some...,meanwhile somewhere america someone getting ch...,3
9996,Who are these anti-vaxx â€œwellness warriorsâ€...,anti vaxx wellness warrior feeding kid placent...,3
9997,@jukegraphics @albertz__ @Luminosity Really wa...,really wanted open premiere go broken almost u...,3
9998,@romeoauerbach Diputado por favor presente una...,diputado por favor presente una iniciativa de ...,0


In [22]:
def neg_pos_neu(data,label):
  sia=SentimentIntensityAnalyzer()
  df['Negative']=df[label].apply(lambda x: sia.polarity_scores(x)['neg'])
  df['Positive']=df[label].apply(lambda x: sia.polarity_scores(x)['pos'])
  df['Netral']=df[label].apply(lambda x: sia.polarity_scores(x)['neu'])
  df['compound']=df[label].apply(lambda x: sia.polarity_scores(x)['compound'])
  return df


In [ ]:
final_df=neg_pos_neu(df,'cleaned')


In [29]:
final_df.head(10)

,Text,cleaned,Label,Negative,Positive,Netral,compound
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,testimony evidence court law state federal mus...,0,0.000,0.000,1.000,0.0000
1,#BadBunny: Como dos gotas de agua: Joven se di...,como do gotas de agua joven se disfraza de bad...,1,0.189,0.000,0.811,-0.5423
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,flagstar bank discloses data breach impacted 1...,0,0.000,0.000,1.000,0.0000
3,Rwanda is set to host the headquarters of Unit...,rwanda set host headquarters united nation dev...,2,0.000,0.310,0.690,0.6597
4,OOPS. I typed her name incorrectly (todayâ€™s ...,oops typed name incorrectly today brave witnes...,0,0.143,0.256,0.602,0.3612
5,It sucks for me since I'm focused on the natur...,suck since focused nature aspect thing environ...,1,0.162,0.260,0.578,0.3400
6,@en_font Treballar a lâ€™obra a partir dels 19...,treballar l obra partir dels 19 anys fin al 50...,1,0.103,0.000,0.897,-0.4767
7,@ShawnTarloff @itsmieu you can also relate thi...,also relate art lot people dismayed starting a...,3,0.102,0.297,0.601,0.6705
8,Social Security. Constant political crises dis...,social security constant political crisis dist...,1,0.412,0.142,0.446,-0.8834
9,@FilmThePoliceLA A broken rib can puncture a l...,broken rib puncture lung lead collapsed lung m...,1,0.321,0.000,0.679,-0.6369


In [31]:
mapping=[]
for i in range(len(final_df)):
  if final_df.loc[i,'compound']<0.0:
    mapping+=[1]
  elif final_df.loc[i,'compound']==0.0:
    mapping+=[0]
  else:
    mapping+=[2]
final_df['Final_label']=mapping
final_df['Final_label'].value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


2    5143
1    3562
0    1295
Name: Final_label, dtype: int64

In [33]:
final_df[['Text','Label','compound','Final_label']].head()

,Text,Label,compound,Final_label
0,@Charlie_Corley @Kristine1G @amyklobuchar @Sty...,0,0.0000,0
1,#BadBunny: Como dos gotas de agua: Joven se di...,1,-0.5423,1
2,https://t.co/YJNiO0p1JV Flagstar Bank disclose...,0,0.0000,0
3,Rwanda is set to host the headquarters of Unit...,2,0.6597,2
4,OOPS. I typed her name incorrectly (todayâ€™s ...,0,0.3612,2


In [35]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 6.4 MB/s 
     |████████████████████████████████| 7.6 MB 47.8 MB/s 
     |████████████████████████████████| 163 kB 71.6 MB/s 


In [36]:
from transformers import pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]